## Convert free-text to JSON representations

In previous exercise with Kindred, PDF labels from Drugs@FDA were used as follows:  
  
1. PDFs were downloaded 
2. PDFs were converted to free-text files.  
3. Free-text files were then separated into different sections for Indications, Contraindications, and Adverse Effects.  (possible LLM task?)
    
This separation was chosen as the types of named entities and relationships would be largely identical, but with different inferred meanings dependent on the section they were present within (this seemed a hard problem to address). Additionally, due to the labels being highly unstructured and designed for visual, human understanding, the conversion from PDF to free-text could be extremely messy. 

In this exercise, Drugs@FDA labels were obtained from the OpenFDA resource and converted from JSON format to a pandas dataframe (excel file). Some of these fields are still messy, but they are now less messy than the previous method and are pre-sectioned.

I am thinking now about how to re-format this dataset for use in HuggingFace models. The aim of this research is to use AI-assisted technology to automatically generate interaction claims from additional, previously under utilized sources of data. In doing so, we can increase data parity between the published state of knowledge and what is available in databases.  
  
**Approach**:  
  
1. Take an existing pretrained model and use train it to perform **entity recognition** of drugs, genes, variants, and phenotypes. 
2. Either (1): identify a suitable model or, (2): train an existing pretrained model further to perform **text summarization** to summarize a label to link drugs, genes, variants, phenotypes within one individual 'claim'. (**relationship classification**??)

In thinking about this, I think the data should eventually be formatted like this:

In [ ]:
# JSON format for Adverse Effects section
{
    "meta": { "label": <identifier>,
                "drug": <drug label>,
                "type": <type of page, i.e. indication, adverse effects, contraindications>,
                "url": <url of label download>,
            },
    "adverse_effects": <free text dump>,
    "tokens": [...],
    "pos_tags": [...], # IDs
    "chunk_tags": [...], # IDs
    "ner_tags": [...], # IDs
    "id": <identifier for data point>

}

## Load Labels using Datasets
Dataset is obtained from OpenFDA json resource. Pre-sectioned labels were imported using the json library and converted to a pandas dataframe. 

In [2]:
from datasets import Dataset
import pandas as pd

df = pd.read_excel('../data/openfda.xlsx').reset_index(drop=True).drop('Unnamed: 0', axis=1)

df


,id,adverse_reactions,indications_and_usage,contraindications,warnings_and_cautions,brand_name
0,b96b2e24-4192-31e2-e053-2a95a90a2356,ADVERSE REACTIONS SECTION The following are di...,INDICATIONS & USAGE SECTION To reduce the deve...,CONTRAINDICATIONS SECTION Amoxicillinfor oral ...,WARNINGS AND PRECAUTIONS SECTION 5.1 Anaphylac...,AMOXICILLIN AND CLAVULANATE POTASSIUM
1,c733ec78-5b43-7a7c-e053-2995a90a2f9a,NaN,Uses For the relief of symptoms associated wit...,NaN,NaN,UNDA 312
2,f21e2f96-6c05-fbe5-e053-2995a90a3948,NaN,Uses Helps prevent sunburn. If used as directe...,NaN,NaN,SUN PROTECT LIP BALM SPF 30
3,de170f72-0f59-4ed3-9715-8bdff49fc5da,NaN,NaN,NaN,NaN,LOSARTAN POTASSIUM AND HYDROCHLOROTHIAZIDE
4,e95e8654-1edf-40f7-8f42-38aacf3dcd08,6 ADVERSE REACTIONS The following clinically s...,1 INDICATIONS AND USAGE Potassium Phosphates I...,4 CONTRAINDICATIONS Potassium Phosphates Injec...,5 WARNINGS AND PRECAUTIONS Serious Cardiac Adv...,Potassium Phosphates
...,...,...,...,...,...,...
215910,2226fb84-50af-45b1-8a9f-72984e331519,ADVERSE REACTIONS The most common adverse reac...,INDICATIONS AND USAGE Carbidopa and levodopa t...,CONTRAINDICATIONS Nonselective monoamine oxida...,NaN,Carbidopa and Levodopa
215911,0776c9e3-a64d-40b2-b22d-aa66a5d4fa76,NaN,"Uses For the temporary relief of burning, irri...",NaN,NaN,REFRESH Optive Mega-3
215912,4ca80987-5d55-4476-a694-c95d3e07cf4c,6 ADVERSE REACTIONS The most serious adverse r...,1 INDICATIONS AND USAGE CREON ® is indicated f...,4 CONTRAINDICATIONS None. None ( 4 ),5 WARNINGS AND PRECAUTIONS Fibrosing colonopat...,Creon
215913,a725ae2b-865e-434c-9281-51f6344b97b9,6 ADVERSE REACTIONS Most common adverse reacti...,1 INDICATIONS AND USAGE Losartan potassium and...,4 CONTRAINDICATIONS Losartan potassium and hyd...,5 WARNINGS AND PRECAUTIONS Hypotension: Correc...,Losartan Potassium and Hydrochlorothiazide


In [3]:
# Datasets will load each column from a dataframe as a dataset but cannot handle single Series objects
f = pd.DataFrame(df, columns=['adverse_reactions'])
dataset = Dataset.from_pandas(f)

In [4]:
# Filter Nonetype and length 
print(f'UNFILTERED: {len(dataset["adverse_reactions"])}')

dataset = dataset.filter(lambda x: x['adverse_reactions'] is not None)
print(f'After NONE CHECK: {len(dataset["adverse_reactions"])}')

test = dataset.filter(lambda x: len(x['adverse_reactions']) < 8000)
print(f'After TITAN LENGTH CHECK: {len(test["adverse_reactions"])}')

dataset = dataset.filter(lambda x: len(x['adverse_reactions']) < 512)
print(f'After ADE LENGTH CHECK: {len(dataset["adverse_reactions"])}')

UNFILTERED: 215915


Filter:   0%|          | 0/215915 [00:00<?, ? examples/s]

After NONE CHECK: 77064


Filter:   0%|          | 0/77064 [00:00<?, ? examples/s]

After TITAN LENGTH CHECK: 59382


Filter:   0%|          | 0/77064 [00:00<?, ? examples/s]

After ADE LENGTH CHECK: 5642


## Perform Named Entity Recognition (NER) for Adverse Reactions Dataset
The model being used for this tag generation is the electramed-small-ADE-DRUG-EFFECT-ner-v3: https://huggingface.co/chintagunta85/electramed-small-ADE-DRUG-EFFECT-ner-v3
      
There is not a very descriptive model card for this on huggingface, but via James it seems like it might be the same as this paper: https://arxiv.org/pdf/2201.01405v2.pdf


In [5]:
# Load ADE-DRUG-EFFECT pipe  
# Paper: https://paperswithcode.com/paper/mining-adverse-drug-reactions-from
# Google Colab: https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/16.Adverse_Drug_Event_ADE_NER_and_Classifier.ipynb

from transformers import pipeline, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
pipe = pipeline("token-classification", model="chintagunta85/electramed-small-ADE-DRUG-EFFECT-ner-v3")

2023-09-28 10:50:16.680609: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Filter and generate tags
def generate_ner_tags(entry):
    return {'ner_tags': pipe(entry['adverse_reactions']) }

dataset = dataset.map(generate_ner_tags)

Map:   0%|          | 0/5642 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
# Inspect
dataset

Dataset({
    features: ['adverse_reactions', 'ner_tags'],
    num_rows: 5642
})

In [8]:
# Inspect
dataset['ner_tags'][3]

[{'end': 7,
  'entity': 'B-EFFECT',
  'index': 1,
  'score': 0.9073781967163086,
  'start': 0,
  'word': 'adverse'},
 {'end': 17,
  'entity': 'I-EFFECT',
  'index': 2,
  'score': 0.9293419122695923,
  'start': 8,
  'word': 'reactions'}]

In [9]:
dataset['adverse_reactions'][3220]

'ADVERSE REACTIONS: Reports of irritation and hypersensitivity to sodium sulfacetamide are uncommon. The following adverse reactions, reported after administration of sterile ophthalmic sodium sulfacetamide, are noteworthy: instances of Stevens-Johnson syndrome and instances of local hypersensitivity which progressed to a syndrome resembling systemic lupus erythematosus; in one case a fatal outcome was reported (see WARNINGS).'

## Rejoin NER Fragments from Adverse Reactions
Once NER has been performed with this model, post-processing must be done to rejoin the chunks where applicable.

In [10]:
def recombine_ner_chunks(entry):
    all_tags = []
    all_words = []
    for tag_set in entry['ner_tags']:
        all_tags.append(tag_set['entity'])
        all_words.append(tag_set['word'])

    i = 0
    chunk_number = []
    for tag in all_tags:
        if tag.startswith('B-') == True:            
            i += 1
            chunk_number.append(i)
        
        if tag.startswith('I-') == True:
            chunk_number.append(i)
    
    position = 0
    rebuilt = []
    word = '' 
    for chunk_indicator in chunk_number:
        current_word = all_words[position]
        try: 
            if chunk_indicator == chunk_number[position + 1]:
                # don't append
                word = word + ' ' + current_word # build word
                position += 1# iterate
            else:
                word = word + ' ' + current_word 
                rebuilt.append(word.strip()) # append
                word = '' # start new word
                position += 1 # iterate
        except:
            word = word + ' ' + current_word
            rebuilt.append(word.strip())


    return({'ner_chunk_indicators': chunk_number, 'individual_ner_words' : all_words, 'ner_chunks': rebuilt })

In [11]:
dataset = dataset.map(recombine_ner_chunks)
dataset

Map:   0%|          | 0/5642 [00:00<?, ? examples/s]

Dataset({
    features: ['adverse_reactions', 'ner_tags', 'ner_chunk_indicators', 'individual_ner_words', 'ner_chunks'],
    num_rows: 5642
})

In [12]:
# Inspect
dataset['ner_chunks']

[['rash'],
 ['adverse reactions section', 'satisfaction', 'vi'],
 ['adverse reactions', 'mo'],
 ['adverse reactions'],
 ['adverse reaction'],
 ['adverse reaction', 'tn 370'],
 ['adverse reactions', 'cp ##da - 1'],
 ['potassium salts'],
 ['allergic sensitization', 'foli ##c acid'],
 ['adverse reactions'],
 ['adverse reaction', 'wal', 'il'],
 ['corticosteroids'],
 ['adverse reaction', 'ald', 'il 60 ##51'],
 ['adverse reactions'],
 ['adverse reactions'],
 [],
 ['adverse reaction', 'ri'],
 ['reactions allergic rash'],
 ['reactions allergic rash'],
 ['vitamin b 12'],
 ['adverse reactions section', 'ave', 'sp ##f'],
 [],
 ['adverse reaction', 'pri'],
 [],
 ['foli ##c acid'],
 ['ny ##stat ##in'],
 ['sham'],
 ['ny ##stat ##in'],
 ['claims satisfaction'],
 ['adverse reactions', 'mo'],
 ['adverse reactions dist', 'oh'],
 ['sp ##f 100 +'],
 ['sham'],
 ['adverse reaction - re'],
 [],
 ['gentamicin'],
 ['lact ##ulose'],
 ['adverse reaction dist'],
 ['adverse reactions'],
 ['adverse reactions', 'top

In [13]:
# IL + zip code getting recognized as an interleukin
dataset[12]

{'adverse_reactions': 'Adverse reaction Distributed by ALDI Inc. Batavia, IL 60510 DOUBLE BUARANTEE REPLACE THE PRODUCT REFUND YOUR MONEY www.ALDI.us PLEASE RECYCLE',
 'ner_tags': [{'end': 7,
   'entity': 'B-EFFECT',
   'index': 1,
   'score': 0.8888990879058838,
   'start': 0,
   'word': 'adverse'},
  {'end': 16,
   'entity': 'I-EFFECT',
   'index': 2,
   'score': 0.8761894702911377,
   'start': 8,
   'word': 'reaction'},
  {'end': 35,
   'entity': 'B-DRUG',
   'index': 5,
   'score': 0.5261055827140808,
   'start': 32,
   'word': 'ald'},
  {'end': 53,
   'entity': 'B-DRUG',
   'index': 13,
   'score': 0.8849818110466003,
   'start': 51,
   'word': 'il'},
  {'end': 56,
   'entity': 'I-DRUG',
   'index': 14,
   'score': 0.3542076349258423,
   'start': 54,
   'word': '60'},
  {'end': 58,
   'entity': 'I-DRUG',
   'index': 15,
   'score': 0.4874825179576874,
   'start': 56,
   'word': '##51'}],
 'ner_chunk_indicators': [1, 1, 2, 3, 3, 3],
 'individual_ner_words': ['adverse', 'reaction', 

In [14]:
# Inspect
dataset[53]

{'adverse_reactions': 'ADVERSE REACTIONS Developing teeth of children under age 6 may become permanently discolored if excessive amounts are repeatedly swallowed. The following adverse reactions are possible in individuals hypersensitive to fluoride: eczema, atopic dermatitis, urticaria, gastric distress, headache, and weakness.',
 'ner_tags': [{'end': 226,
   'entity': 'B-DRUG',
   'index': 35,
   'score': 0.9134745001792908,
   'start': 218,
   'word': 'fluoride'}],
 'ner_chunk_indicators': [1],
 'individual_ner_words': ['fluoride'],
 'ner_chunks': ['fluoride']}

TO DO: quantify different occurences and see if anything pops up? also post processing still to remove the ##s? get rid of occurences of the word adverse reaction(s)

In [15]:
dataset_df = dataset.to_pandas()

In [19]:
dataset_df['ner_chunks']

0                                              [rash]
1       [adverse reactions section, satisfaction, vi]
2                             [adverse reactions, mo]
3                                 [adverse reactions]
4                                  [adverse reaction]
                            ...                      
5637                                [potassium salts]
5638                       [hyper ##thy ##roid ##ism]
5639                                               []
5640                                  [foli ##c acid]
5641                                               []
Name: ner_chunks, Length: 5642, dtype: object

## Evaluate Performance TODO
Figure out a way using the model's score to assess performance

In [66]:
dataset_50 = 
# dataset_80 = 
# dataset_95 =

# manually label dataset? 100 sentences? 32 lowball


Filter:   0%|          | 0/647 [00:00<?, ? examples/s]

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/m6/b6y4g9114836jky8p81w12mchscvrj/T/ipykernel_24266/3357191444.py:1 in <cell │
│ line: 1>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/m6/b6y4g9114836jky8p81w12mchscvrj/T/ipykernel_24266/3357191444.py'          │
│ /Users/mjc014/.pyenv/versions/3.10.4/lib/python3.10/site-packages/datasets/arrow_dataset. │
│ py:543 in wrapper                                                                         │
│                                                                                           │
│    540 │   │   │   "output_all_columns": self._output_all_columns,                        │
│    541 │   │   }                                                                          │
│    542 │   │   # apply actual function                                                    │
│ ❱  543 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)         │
│    544 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) el │
│    545 │   │   # re-apply format to the output                                            │
│    546 │   │   for dataset in datasets:                                                   │
│                                                                                           │
│ /Users/mjc014/.pyenv/versions/3.10.4/lib/python3.10/site-packages/datasets/fingerprint.py │
│ :511 in wrapper                                                                           │
│                                                                                           │
│   508 │   │   │                                                                           │
│   509 │   │   │   # Call actual function                                                  │
│   510 │   │   │                                                                           │
│ ❱ 511 │   │   │   out = func(dataset, *args, **kwargs)                                    │
│   512 │   │   │                                                                           │
│   513 │   │   │   # Update fingerprint of in-place transforms + update in-place history o │
│   514                                                                                     │
│                                                                                           │
│ /Users/mjc014/.pyenv/versions/3.10.4/lib/python3.10/site-packages/datasets/arrow_dataset. │
│ py:3600 in filter                                                                         │
│                                                                                           │
│   3597 │   │   if len(self) == 0:                                                         │
│   3598 │   │   │   return self                                                            │
│   3599 │   │                                                                              │
│ ❱ 3600 │   │   indices = self.map(                                                        │
│   3601 │   │   │   function=partial(                                                      │
│   3602 │   │   │   │   get_indices_from_mask_function, function, batched, with_indices, i │
│   3603 │   │   │   ),                                                                     │
│                                                                                           │
│ /Users/mjc014/.pyenv/versions/3.10.4/lib/python3.10/site-packages/datasets/arrow_dataset. │
│ py:578 in wrapper                                                                         │
│                                                   

In [ ]:
dataset_50 = []
dataset_80 = []
dataset_95 = []

for entry in dataset:
    for ner_tags in entry['ner_tags']:
        try:
            if ner_tags[0]['score'] > 0.50:
                dataset_50.append()
            if ner_tags[0]['score'] > 0.80:
                print(ner_tags[0])
            if ner_tags[0]['score'] > 0.95:
                print(ner_tags[0])

            else:
                pass
        except:
            pass